In [8]:
# =========================
# 🔹 TRAIN + VALIDATE + TEST CUSTOM CNN (GRAYSCALE FIXED)
# =========================

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image
import os, glob

# ---------------------------- Settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_EPOCHS = 20           # Increase epochs for better learning
BATCH_SIZE = 32
LEARNING_RATE = 0.001
IMAGE_SIZE = 64           # Match your model input

data_dir = r"D:\DeepTech\data"
train_folder = os.path.join(data_dir, "train")
val_folder   = os.path.join(data_dir, "valid")
test_folder  = os.path.join(data_dir, "test")

# ---------------------------- Data Loaders (Grayscale + Augmentation)
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to 1 channel
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to 1 channel
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.ImageFolder(train_folder, transform=train_transforms)
val_dataset   = datasets.ImageFolder(val_folder, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE)

class_names = train_dataset.classes
num_classes = len(class_names)

# ---------------------------- Define Custom CNN (1-channel input)
class MyCustomCNN(nn.Module):
    def __init__(self, num_classes=num_classes):
        super(MyCustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # 1 channel
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)      # IMAGE_SIZE/4 = 16
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MyCustomCNN().to(device)

# ---------------------------- Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
best_val_acc = 0

# ---------------------------- Training Loop
for epoch in range(NUM_EPOCHS):
    model.train()
    correct, total, running_loss = 0, 0, 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = out.argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    train_acc = 100 * correct / total
    train_loss = running_loss / len(train_loader)

    # ---- Validation ----
    model.eval()
    correct, total, val_loss = 0, 0, 0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out, y)
            val_loss += loss.item()
            preds = out.argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    val_acc = 100 * correct / total
    val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            "model_state_dict": model.state_dict(),
            "class_names": class_names,
            "val_acc": val_acc
        }, "best_cnn_phase1.pth")

print("✅ Training done. Best Val Acc:", best_val_acc)

# ---------------------------- Test Function
def predict_image(img_path, model, class_names):
    model.eval()
    img = Image.open(img_path).convert("L")  # Ensure 1 channel
    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(img_tensor)
        probs = torch.softmax(logits, dim=1)[0]
        max_prob, idx = torch.max(probs, 0)
    print(f"Image: {os.path.basename(img_path)} | "
          f"Predicted Class: {class_names[idx]} | Confidence: {max_prob.item()*100:.2f}%")

# ---------------------------- Test All Images in Test Folder
checkpoint = torch.load("best_cnn_phase1.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
for img_path in glob.glob(os.path.join(test_folder, "*.*")):
    predict_image(img_path, model, class_names)


Epoch [1/20] | Train Loss: 0.5161 | Train Acc: 83.16% | Val Loss: 0.1953 | Val Acc: 93.16%
Epoch [2/20] | Train Loss: 0.1947 | Train Acc: 93.49% | Val Loss: 0.1944 | Val Acc: 93.02%
Epoch [3/20] | Train Loss: 0.1220 | Train Acc: 96.00% | Val Loss: 0.0855 | Val Acc: 96.87%
Epoch [4/20] | Train Loss: 0.0811 | Train Acc: 97.88% | Val Loss: 0.0731 | Val Acc: 97.72%
Epoch [5/20] | Train Loss: 0.0625 | Train Acc: 98.04% | Val Loss: 0.1216 | Val Acc: 96.30%
Epoch [6/20] | Train Loss: 0.0543 | Train Acc: 98.38% | Val Loss: 0.0606 | Val Acc: 98.01%
Epoch [7/20] | Train Loss: 0.0450 | Train Acc: 98.63% | Val Loss: 0.0515 | Val Acc: 98.43%
Epoch [8/20] | Train Loss: 0.0351 | Train Acc: 98.97% | Val Loss: 0.0808 | Val Acc: 97.58%
Epoch [9/20] | Train Loss: 0.0295 | Train Acc: 99.05% | Val Loss: 0.0396 | Val Acc: 98.58%
Epoch [10/20] | Train Loss: 0.0290 | Train Acc: 99.18% | Val Loss: 0.0400 | Val Acc: 98.86%
Epoch [11/20] | Train Loss: 0.0194 | Train Acc: 99.36% | Val Loss: 0.0559 | Val Acc: 98.1

In [16]:
# =========================
# 🔹 TEST SINGLE IMAGE
# =========================

import torch
from torchvision import transforms
from PIL import Image
import os

# ---------------------------- Settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = 64  # Must match training
img_path = r"D:\DeepTech\data\train\Local\image_LOC_32014_png_jpg.rf.291c9481f3abfc2a417e7b99b3cea11a.jpg"

# ---------------------------- Load checkpoint
checkpoint = torch.load("best_cnn_phase1.pth", map_location=device)
class_names = checkpoint['class_names']

# Define the same model architecture
import torch.nn as nn
import torch.nn.functional as F

class MyCustomCNN(nn.Module):
    def __init__(self, num_classes=len(class_names)):
        super(MyCustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # grayscale input
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model and load weights
model = MyCustomCNN().to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# ---------------------------- Prediction
def predict_single_image(img_path, model, class_names):
    img = Image.open(img_path).convert("L")  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(img_tensor)
        probs = torch.softmax(logits, dim=1)[0]
        max_prob, idx = torch.max(probs, 0)
    print(f"Image: {os.path.basename(img_path)}")
    print(f"Predicted Class: {class_names[idx]}")
    print(f"Confidence: {max_prob.item() * 100:.2f}%")

# Run prediction
predict_single_image(img_path, model, class_names)


Image: image_LOC_32014_png_jpg.rf.291c9481f3abfc2a417e7b99b3cea11a.jpg
Predicted Class: Local
Confidence: 100.00%


In [37]:
# =========================
# 🔹 TEST IMAGE WITH INVALID CHECK
# =========================

import torch
from torchvision import transforms
from PIL import Image
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_SIZE = 64
CONFIDENCE_THRESHOLD = 0.7  # 70% threshold

# Load saved model checkpoint
checkpoint = torch.load("best_cnn_phase1.pth", map_location=device)
class_names = checkpoint['class_names']

# Define the CNN (same as training)
import torch.nn as nn
import torch.nn.functional as F

class MyCustomCNN(nn.Module):
    def __init__(self, num_classes=len(class_names)):
        super(MyCustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model and load weights
model = MyCustomCNN().to(device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# ---------------------------- Prediction function with invalid check
def predict_image_with_invalid(img_path, model, class_names, threshold=CONFIDENCE_THRESHOLD):
    img = Image.open(img_path).convert("L")  # grayscale
    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(img_tensor)
        probs = torch.softmax(logits, dim=1)[0]
        max_prob, idx = torch.max(probs, 0)

    if max_prob.item() < threshold:
        print(f"Image: {os.path.basename(img_path)} | Prediction: INVALID / Unknown image")
    else:
        print(f"Image: {os.path.basename(img_path)} | Predicted Class: {class_names[idx]} | Confidence: {max_prob.item()*100:.2f}%")

# ---------------------------- Test single or multiple images
test_images = [
    r"D:\DeepTech\data\train\Center\image_Center_12009_png_jpg.rf.f507dcdd0e47b802409836643343b63d.jpg"  # unrelated image
]

for img_path in test_images:
    predict_image_with_invalid(img_path, model, class_names)


Image: image_Center_12009_png_jpg.rf.f507dcdd0e47b802409836643343b63d.jpg | Predicted Class: Center | Confidence: 99.99%


In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Load checkpoint
checkpoint = torch.load("best_cnn_phase1.pth", map_location="cpu")
class_names = checkpoint['class_names']

class MyCustomCNN(nn.Module):
    def __init__(self, num_classes=len(class_names)):
        super(MyCustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = MyCustomCNN()
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Export to ONNX
dummy_input = torch.randn(1, 1, 64, 64)
onnx_file = "wafer_model.onnx"
torch.onnx.export(
    model, dummy_input, onnx_file,
    input_names=["input"],
    output_names=["output"],
    opset_version=11
)
print(f"✅ Model exported to ONNX: {onnx_file}")


W0208 11:01:30.739000 3752 site-packages\torch\onnx\_internal\exporter\_compat.py:125] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0208 11:01:31.357000 3752 site-packages\torch\onnx\_internal\exporter\_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, aligned: 'bool' = False). Treating as an Input.
W0208 11:01:31.360000 3752 site-packages\torch\onnx\_internal\exporter\_schemas.py:455] Missing annotation for parameter 'boxes' from (input, boxes, output_size: 'Sequence[int]', spatial_scale

[torch.onnx] Obtain model graph for `MyCustomCNN([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `MyCustomCNN([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


c:\Users\pranav shankar\AppData\Local\Programs\Python\Python312\Lib\copyreg.py:99: FutureWarning: `isinstance(treespec, LeafSpec)` is deprecated, use `isinstance(treespec, TreeSpec) and treespec.is_leaf()` instead.
  return cls.__new__(cls, *args)
The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 11).
Failed to convert the model to the target version 11 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "c:\Users\pranav shankar\AppData\Local\Programs\Python\Python312\Lib\site-packages\onnxscript\version_converter\__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pranav shankar\AppData\Local\Programs\Python\Python312\Lib\site-packages\onnxscript\version_converter\_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
✅ Model exported to ONNX: wafer_model.onnx


In [36]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("wafer_model_tf")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open("wafer_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Model converted to TFLite: wafer_model.tflite")


OSError: SavedModel file does not exist at: wafer_model_tf\{saved_model.pbtxt|saved_model.pb}